# Preparation

## import tools

In [ ]:
import os, numpy as np
import histogram.hdf as hh
import mcvine, mcvine.components, mcvine.run_script

from matplotlib import pyplot as plt
%matplotlib notebook

## work dir

In [ ]:
workdir = os.path.expanduser("~/simulations/mcvine/demo/2.adjust_parameters")
!mkdir -p {workdir}
%cd {workdir}

# Explore

## create a script containing an instrument method with a parameter

In [ ]:
%%file myinstrument.py
import mcvine, mcvine.components
def instrument(E0=60):
    instrument = mcvine.instrument()
    source = mcvine.components.sources.Source_simple('source', E0=E0)
    instrument.append(source, position=(0,0,0))
    monitor = mcvine.components.monitors.E_monitor('monitor', filename='IE.dat', Emin=0, Emax=130)
    instrument.append(monitor, position=(0,0,1))
    return instrument

## run simulation using MPI, setting the parameter

In [ ]:
%%time
mcvine.run_script.run_mpi('myinstrument.py', "out", 1e6, nodes=10, overwrite_datafiles=True, E0=70)

## check simulation results

In [ ]:
IE = hh.load('./out/IE.h5')
plt.figure()
plt.plot(IE.energy, IE.I)

# Run a series of simulations, varying parameters

## simulate

In [ ]:
for E0 in np.arange(20, 120, 30.):
    print E0
    mcvine.run_script.run_mpi('myinstrument.py', "work", 1e6, nodes=10, overwrite_datafiles=True, E0=E0)
    !cp work/IE.h5 IE_E_{E0}.h5
    !rm -rf work
    continue

## check results

In [ ]:
ls -tl IE_E*.h5

In [ ]:
plt.figure()
for E0 in np.arange(20, 120, 30.):
    h = hh.load('IE_E_{}.h5'.format(E0))
    plt.plot(h.energy, h.I, label="={}".format(E0))
    continue
plt.legend()
plt.xlabel("E(meV)")

# Debug simulation script

create yaml file to store the parameters

In [ ]:
%%file params.yaml
E0: 100

run simulation

In [ ]:
!python -m "mcvine.run_script" myinstrument.py \
    --workdir debug --overwrite_datafiles \
    --ncount 1000000 --buffer_size 20000 \
    --run-pps \
    --additional-kargs params.yaml

In [ ]:
IE = hh.load('./debug/IE.h5')
plt.figure()
plt.plot(IE.energy, IE.I)